# Data Set Creation 

__Raw Data__: 
- MovieLens
    - Movies: movies with ID numbers, title, release year, genre
    - Ratings: user Ids with ratings for movieIds. 0.5-5 starts on 0.5 increments
    - Tags: all possible genome tags 
    - Relevance: list of all genome tags for each movie with relevance score (0-100)
    - Links: movieId with corresponding imdbId
- IMDB: movie, imdbId, actors, directors, description, genres, production company, year of release, country. Plus additional columns not using for content vectors: duration, language, income, budget, user reviews, critics reviews
    - Limited amount of missing data for director, actors, country, description, production company. Small numbers. Plus can still use the non-missing data for a given movie to build its content vector/profile

__Data Creation__: 
- Merge in IMDB data to get additional metadata 
    - First merge movies + links to get imdbId 
    - Lose 17140 movies that aren't in IMDB dataset. Could not find larger dataset that had imdbIDs for merging
- Limit ratings data to movies in data post imdb merge
    - Do not have to limit overall movies to ones with ratings. Can recommend movies without ratings based on their content profile
    - Save version of ratings data with 25% random user subsample to use in UI to improve processing time
- Calculate aggregate rating data by movie for use in evaluations, sorting recommendations etc. Save as parquet. 
    - Average rating
    - Count of ratings
- Define decade of release
- Merge genome tags + relevance scores for each movie
    - 28% of movies have tags
- Clean genre: IMDB and MovieLens have slightly different lists. Take union of two. 
- Text cleaning for description and genome tags
- Find top 5 relevant genome tags for each movie 
- Find top 5 TFIDF tokens for each movie for following fields:
    - Description
    - Genome Tags (relevance > 75%)
    - Text: Tags + Description combined into one field 
- Separate delimited fields into lists (ex multiple actors per movie in format actor1|actor2)
- For each metadata, find counts (# of movies) per value and exclude values that are only in 1 movie --> not useful for content profile comparison and decrease memory
- One hot encode content features
    - For actors, too many values to fit all encodings in memory. Get top 3 actors in each movie based on number of other movies they have been in. Then check again and exclude actors only in 1 movies out of this subset
    - Create multiple versions of data with different combinations of feature vectors4
    
    
__Outputted Datasets__: 
- ratings_sample.parq: sampled ratings dataset + nonIMDB movies excluded 
- movies_ratings.parq: average and total reviews for each movie
- movies_processed.parq: processed movies dataframe before one hot encoding and exclusions necessary for that encoding. Used for data display in UI (additional featuers added in recommendation_data_display) and for EDA (movies_EDA)
- processed_df + <>.parq ; processed_df + <>.npz ; sparse_metadata + <>
    - Variety of datasets with different one hot encoded features for building content model 
   

In [1]:
import pandas as pd
import os
import numpy as np
from scipy.sparse import csc_matrix
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as datetime
import operator
import fastparquet
from sklearn.feature_extraction.text import TfidfVectorizer
import pickle
import scipy
import nltk
from nltk.corpus import stopwords
import spacy
import re
#nlp = spacy.load('en')
#nltk.download('stopwords')
sw = stopwords.words("english")
import math
import random

In [2]:
links = pd.read_csv('data/ml-25m/links.csv')
movies = pd.read_csv('data/ml-25m/movies.csv')
ratings = pd.read_csv('data/ml-25m/ratings.csv')
tags = pd.read_csv('data/ml-25m/genome-tags.csv')
relevance = pd.read_csv('data/ml-25m/genome-scores.csv')

### Merge all Data (including IMDB dataset)
- Movies + Links. On movieId 
- Merge in IMDB. On imdbId from links 

Searched kaggle for a larger set and did not find one that also had IMDB IDs    
Drop movies that are not in IMDB because need metadata to recommend

In [3]:
# IMDB dataset
imdb_movies = pd.read_csv('data/imdb/IMDb movies.csv')

# standardize IMDB IDs
imdb_movies['imdbId'] = imdb_movies.imdb_title_id.str.split('tt').str[1]
imdb_movies.imdbId = pd.to_numeric(imdb_movies.imdbId)

In [4]:
num_movies = len(movies)
# links (has imdb rating) + movies
df = pd.merge(links, movies, on = 'movieId')
# merge with imdb data
# INNER merge so that only get movies that are in movielens data + have content from IMDB 
df = pd.merge(df, imdb_movies, on = 'imdbId', how = 'inner')

# titles are different in movielens vs imdb because imdb in the original language whereas movielens all english translated
df = df.rename(columns = {'title_x':'title_eng', 'title_y':'title_orig'})

new_num_movies = len(df)

In [5]:
# limit to relevant columns
df = df[['movieId', 'title_eng', 'year', 'genre', 'genres', 'director', 'actors', 'country', 
         'description', 'production_company']]

## Missing Data Post Merge

In [6]:
# number of movies lost in merge
num_movies - new_num_movies

17410

In [7]:
new_num_movies

45013

In [8]:
df.isnull().sum()

movieId                  0
title_eng                0
year                     0
genre                    0
genres                   0
director                13
actors                  20
country                 10
description            404
production_company    1230
dtype: int64

## Limit Ratings Data to Movies in IMDB + Save Ratings Data
To build user profiles based on ratings, need movie content info    
Do NOT however have to limit movie data to movies in ratings. Can recommend movies with no ratings.    
  
- ratings_sample: ratings data for evaluation    
- ratings_sample_user add: random 25% sample of users for loading into UI app to increase performance. Most people will manually enter profiles, but want some preloaded to show that functionality as well. This will also be the dataset that we add the entered profiles onto.
    - Subsample based on users rather than observations so that each user has their full set of ratings to get recommendations based on 

In [9]:
# ratings data for evaluation
ratings = ratings[ratings.movieId.isin(df.movieId.unique())]
ratings.to_parquet('processed_files/ratings_sample.parq', engine = 'fastparquet', compression = 'GZIP', index = False)

# ratings data for app
random.seed(1)
random_users = random.sample(list(ratings.userId.unique()), int(len(ratings.userId.unique())*.25))
ratings_useradd = ratings[ratings.userId.isin(random_users)]
ratings_useradd = ratings_useradd.drop(columns = ['timestamp'])
ratings_useradd.to_parquet('processed_files/ratings_sample_useradd.parq', engine = 'fastparquet', compression = 'GZIP', index = False)

### Calculate count and average of ratings by movie 

In [10]:
def movie_ratings_agg(df_full, var):
    
    ratings_gb = df_full.groupby(var).rating.mean().to_frame()
    ratings_gb.columns = ['avg']
    ratings_gb['id'] = ratings_gb.index
    ratings_gb = pd.merge(ratings_gb, df[['movieId', 'title_eng']], left_on = 'id', right_on = 'movieId')

    # many movies with average = 5, but many of these have only been rated a few times
    # merge and secondary sort by number of ratings
    num_ratings = df_full.groupby('movieId').rating.count().to_frame()
    num_ratings.columns = ['cnt']
    num_ratings['id'] = num_ratings.index

    ratings_gb = pd.merge(ratings_gb, num_ratings, on = 'id')

    # sort on rating and number of ratings
    ratings_gb = ratings_gb.sort_values(['avg',  'cnt'], ascending = True)
    
    return ratings_gb

__Most highly rated movies: Average with at least 100 ratings__

In [11]:
# Merge rating dataset with movies
# lose some movies in merge because not in imdb movie set 
df_full = pd.merge(df, ratings, on = 'movieId')

# get aggregated movie ratings
movie_ratings = movie_ratings_agg(df_full, 'movieId')

_Save df for evaluations, sorting of other recommendations etc._

In [12]:
movie_ratings.to_parquet('processed_files/movies_ratings.parq', engine = 'fastparquet', compression = 'GZIP')

## Define Release Decade

In [13]:
# decade to filter by 
def rounddown(row):
    return int(math.floor(row / 10.0)) * 10

df['decade'] = df.year.apply(lambda row: rounddown(row))
# convert to string for filtering with lists
df.decade = df.decade.apply(str)

## Genome Tagging Data: Merge Tags + Relevance 
Two perspectives for metadata (created later):
- Top 5 tags per movie in terms of relevance score
- Top 5 tags per movie in terms of TF-IDF score

In [14]:
# merge tags and relevance scores
tags = pd.merge(tags, relevance, on = 'tagId')

# exclude movies not in main df
tags = tags[tags.movieId.isin(df.movieId.unique())]

In [15]:
# percent of movies with tags
tags.movieId.nunique() / df.movieId.nunique()

0.2766978428454002

## Clean Genre Variable
- IMDB and MovieLens sometimes have a different genre list for the same movie. Take the union of both lists to get the max number of genres/information.  
- MovieLens: genres
- IMDB: genre



In [16]:
# movielens missing value
df.genres = np.where(df.genres == '(no genres listed)', np.nan, df.genres)

# convert into sets and take union 
df.genre = df.genre.str.split(', ')
df.genres = df.genres.str.split('|')

# when missing in movielens, replace to list rather than nan 
df.genres = df.genres.apply(lambda d: d if isinstance(d, list) else [])

df.genre = df.genre.apply(set)
df.genres = df.genres.apply(set)

df['genres_all'] = df.apply(lambda x: x['genre'].union(x['genres']), axis=1)
df.genres_all = df.genres_all.apply(list)

## Convert Delimiters into Lists

In [17]:
for var in ['director', 'actors', 'country']:
    df[var + '_lst'] = df[var].str.split(', ')
    df[var + '_lst'] = df[var + '_lst'].apply(lambda d: d if isinstance(d, list) else [])

## Text Cleaning: Description, Genome Tags
- Downcase, remove non-ASCII, remove punctuation, trim white space
- Tokenize
- Remove stop words (except no and not - commonly  used as negation in tags)
- Lemmatize
- Construct token lists into "sentences" separated by spaces 

In [18]:
# clear up memory 
del imdb_movies
del movies
del links

In [19]:
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in text]

In [20]:
# stopwords: exclude no and not -- used in tags as negations (ex not funny vs funny ; no plot vs plot)
sw = [w for w in sw if w not in ['not', 'no']]

In [21]:
def text_prep(df, var, clean = True):
    
    ## clean text
    
    if clean:
    
        # replace missing with empty string
        df[var] = np.where(df[var].isnull(), '', df[var])

        # lower case
        df[var] = df[var].str.lower()
        # remove non-ascii
        df[var] = df[var].apply(lambda x: ''.join([" " if ord(i) < 32 or ord(i) > 126 else i for i in str(x)])) 
        # replace dash with space
        df[var] = df[var].apply(lambda x: re.sub('-', ' ', x))
        # remove punctuation. Replace with '' so keep contractions together
        df[var] = df[var].apply(lambda x: re.sub(r'[^\w\s]','', x))
        # remove double and triple spaces
        df[var] = df[var].apply(lambda x: re.sub(' +', ' ', x)) 
        # remove white space trailing/leading
        df[var] = df[var].apply(lambda x: str(x).strip()) 
        
        # tokenize
        df[var] = df[var].apply(lambda row: row.split(' '))
    
    ## text processing

    # remove stop words
    df[var] = df[var].apply(lambda row: [w for w in row if w not in sw])

    # lemmatize. Do NOT remove proper nouns/do POS tagging
    df[var] = df[var].apply(lemmatize_text)

    # reconstruct sentences
    df[var] = df[var].apply(lambda row: ' '.join(row))
    
    return df

## Top 5 Relevant Tags for each movie 
- Clean text for tags 
- Group by cleaned version and take maximum relevance score. Thus dropping "duplicate" tags like zombie and zombies
    - Manually define group of duplicates around based on a book
- For each movie, get the top 5 relevant cleaned tags 

__Drop "Duplicate" Tags__

In [22]:
# clean list of tags to find duplicate tags (ex zombie, zombies ; based on a book, based on book). Keep original version
unique_tags = tags[['tag']].drop_duplicates()
unique_tags['tag_cleaned'] = unique_tags.tag
unique_tags = text_prep(unique_tags, 'tag_cleaned', clean = True)

# manual cleaning: very common tag with several versions 
unique_tags.tag_cleaned = np.where(unique_tags.tag_cleaned.isin(['literary adaptation', 'adapted from book',
                                                                 'adaptation', 'based book', 'book', 'adapted frombook']),
                                   'adapted from book', unique_tags.tag_cleaned)

# Merge with original tag data. For each movie, keep duplicate token with higher relevance score 
tags = pd.merge(tags, unique_tags, on = 'tag')
tags = tags.groupby(['movieId', 'tag_cleaned']).relevance.max()
tags = tags.reset_index()

__Top 5 Relevant Genome Tags__

In [23]:
# top 5 relevant tags for each movie
tags_top5 = tags.sort_values(['movieId', 'relevance'], ascending = False)
tags_top5 = tags_top5.groupby('movieId').head(5)
tags_top5 = tags_top5.groupby('movieId').tag_cleaned.unique().to_frame()
tags_top5.columns = ['tags_rel']

# merge with dataframe
df = pd.merge(df, tags_top5, left_on = 'movieId', right_index = True, how = 'left')

# combine multiple word tags into a single token 
df.tags_rel = np.where(df.tags_rel.isnull(), '', df.tags_rel)
df.tags_rel = df.tags_rel.apply(lambda row: [i.replace(' ', '') for i in row])

## Top 5 TFIDF: Description, Tags, Text (Tags + Description) 
- TF-IDF vectors
- For each movie, get top 5 tfidf vectors 
- Save to pickle (long process especially if description involved)    
    
For tags, only consider for tfidf if above 75% relevance score for a movie

In [24]:
def tfidf(df, var, filename):
    
    ## TF-IDF
    tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 1),min_df=0, stop_words='english')
    tfidf_matrix = tf.fit_transform(df[var])

    # For each movie, list of columns with a term in tf-idf matrix 
    nonzero_cols = np.split(tfidf_matrix.indices, tfidf_matrix.indptr)[1:-1]
    
    # get top 5 tf-idf tokens for each movie. output is column number in tfidf
    top5_cols_lst = []
    for i in range(tfidf_matrix.shape[0]):
        top5_cols = np.argsort(tfidf_matrix[0,list(nonzero_cols[i])].toarray()[0])[::-1][:5]
        top5_cols = nonzero_cols[i][top5_cols]
        top5_cols_lst.append(top5_cols)

    # get words for those column numbers
    top5_cols_lst_names = [[tf.get_feature_names()[i] for i in j] for j in top5_cols_lst]

    with open(filename + '.pkl', 'wb') as f:
        pickle.dump(top5_cols_lst_names, f)
        
    return top5_cols_lst_names

__Top 5 TFIDF Description__

In [25]:
# clean description text
df = text_prep(df, 'description')

In [26]:
#top5_cols_lst_names = tfidf(df, 'description', "processed_files/tfidf_top5_cols") -- 1.5 hours. Load from pickle instead. 

In [27]:
with open('processed_files/tfidf_top5_cols.pkl', 'rb') as f:
    top5_cols_lst_names = pickle.load(f)
    
top5_cols_lst_names = pd.Series(top5_cols_lst_names).to_frame()
top5_cols_lst_names.columns = ['desc_top5']

# merge top5 tokens with df
df = pd.merge(df, top5_cols_lst_names, left_index = True, right_index = True)

__Top 5 TFIDF Genome Tags__

In [28]:
# limit to relevance >= 75% and merge in 
tags_movies = tags[tags.relevance > 0.75].groupby('movieId').tag_cleaned.unique()
df = pd.merge(df, tags_movies, on = 'movieId', how = 'left')

# combine multiple word tags into a single token 
df.tag_cleaned = np.where(df.tag_cleaned.isnull(), '', df.tag_cleaned)
df.tag_cleaned = df.tag_cleaned.apply(lambda row: [i.replace(' ', '') for i in row])

# combine into sentences for tfidf
df.tag_cleaned = df.tag_cleaned.apply(lambda row: ' '.join(row))

In [29]:
top5_cols_lst_names_tags = tfidf(df, 'tag_cleaned', "processed_files/tfidf_top5_tags_cols")

In [30]:
with open('processed_files/tfidf_top5_tags_cols.pkl', 'rb') as f:
    top5_cols_lst_names_tags = pickle.load(f)
    
top5_cols_lst_names_tags = pd.Series(top5_cols_lst_names_tags).to_frame()
top5_cols_lst_names_tags.columns = ['tag_top5']

# merge top5 tokens with df
df = pd.merge(df, top5_cols_lst_names_tags, left_index = True, right_index = True)

__Top 5 TFIDF Description + Tags__    
Only ~25% of movies have tags

In [31]:
df['text'] = df.tag_cleaned + df.description
df.text = np.where(df.text.isnull(), '', df.text)

In [32]:
#top5_cols_lst_names_text = tfidf(df, 'text', "tfidf_top5_text_cols")  # 2 hours

In [33]:
with open('processed_files/tfidf_top5_text_cols.pkl', 'rb') as f:
    top5_cols_lst_names_text = pickle.load(f)
    
top5_cols_lst_names_text = pd.Series(top5_cols_lst_names_text).to_frame()
top5_cols_lst_names_text.columns = ['text_top5']

# merge top5 tokens with df
df = pd.merge(df, top5_cols_lst_names_text, left_index = True, right_index = True)

FileNotFoundError: [Errno 2] No such file or directory: 'processed_files/tfidf_top5_text_cols.pkl'

## Save Dataframe for Display in UI and EDA
Before one hot encoding and filtering out actors etc.   
Keep cleaned version of tags for display     

In [34]:
df_save = df.copy()
df_save['tag'] = df_save.tag_cleaned.apply(lambda row: row.split(' '))
df_save = df_save.drop(columns = ['genre', 'genres'])
df_save.to_parquet('processed_files/movies_processed.parq', engine = 'fastparquet', compression = 'GZIP')

## For each delimited variable, get dataframe of occurrence counts

Ex: Tom Hanks | Leonardo Dicaprio      
    Tom Hanks        
    Angelina Jolie    
    
Tom Hanks 2       
Leonardo Dicaprio 1       
Angeline Jolie 1         

In [35]:
def cat_list_expand(df, var):
    
    # expand lists such that one entry per row 
    expanded = df[[var, 'movieId']]
    expanded = pd.DataFrame({
        col:np.repeat(expanded[col].values, expanded[var].str.len()) for col in expanded.columns.drop(var)}
    ).assign(**{var:np.concatenate(expanded[var].values)})[expanded.columns]

    return expanded

In [36]:
def delimited_count(df, var, new_var):
    # expand lists such that one entry per row 
    expanded = cat_list_expand(df, var)

    # groupby and count # of occurrences
    counts = expanded.groupby(var)[var].count().to_frame()
    counts.columns = ['cnt']
    # sort
    counts = counts.sort_values(['cnt'])
    counts[new_var] = counts.index
    
    return counts

In [38]:
genre_counts = delimited_count(df, 'genres_all', 'genres')
actors_counts = delimited_count(df, 'actors_lst', 'actors')
directors_counts = delimited_count(df, 'director_lst', 'director')
country_counts = delimited_count(df, 'country_lst', 'country')
desc_counts = delimited_count(df, 'desc_top5', 'desc')
tag_counts = delimited_count(df, 'tag_top5', 'tag')
tag_rel_counts = delimited_count(df, 'tags_rel', 'tag_rel')
text_counts = delimited_count(df, 'text_top5', 'text')

###  Lists of values to exclude if only in 1 movie  
Not helpful for identifying similar movies 

__Directors__

In [39]:
print(len(directors_counts[directors_counts.cnt == 1]) / len(directors_counts))

directors_exclude = directors_counts[directors_counts.cnt == 1].director.unique()

0.5920960909432141


__Actors__

In [40]:
print(len(actors_counts[actors_counts.cnt == 1]) / len(actors_counts))

actors_exclude = actors_counts[actors_counts.cnt == 1].actors.unique()

0.6667830620373791


__Countries__

In [41]:
print(len(country_counts[country_counts.cnt == 1]) / len(country_counts))

country_exclude = country_counts[country_counts.cnt == 1].country.unique()

0.17142857142857143


__Production Companies__

In [42]:
prod_counts = df.groupby('production_company').production_company.count().to_frame()
prod_counts.columns = ['cnt']
# sort
prod_counts = prod_counts.sort_values(['cnt'])
prod_counts['production_company'] = prod_counts.index
    
print(len(prod_counts[prod_counts.cnt == 1]) / len(prod_counts))

prod_exclude = prod_counts[prod_counts.cnt == 1].index.unique()

0.7027213227695488


__Description Tokens__

In [43]:
print(len(desc_counts[desc_counts.cnt == 1]) / len(desc_counts))

desc_exclude = desc_counts[desc_counts.cnt == 1].desc.unique()

0.5089156626506024


__Tag Tokens__

In [44]:
print(len(tag_counts[tag_counts.cnt == 1]) / len(tag_counts))

tags_exclude = tag_counts[tag_counts.cnt == 1].tag.unique()

0.04196519959058342


__Tag Relevance__

In [45]:
print(len(text_counts[text_counts.cnt == 1]) / len(text_counts))

text_exclude = text_counts[text_counts.cnt == 1].text.unique()

0.5175674142430317


__Text Tokens__

In [45]:
print(len(tag_rel_counts[tag_rel_counts.cnt == 1]) / len(tag_rel_counts))

tags_rel_exclude = tag_rel_counts[tag_rel_counts.cnt == 1].tag_rel.unique()

0.03736479842674533


## One Hot Encode Content Variables  
- Exclude values that are only in 1 movie: not useful for recommendations (+ reduce memory)   

Additional cleaning for actors: very large quantity if include all actors in all movies. Too much memory.   
- Get top 3 actors from each movies. "Top" defined by how many movies they have appeared in 
- Check again for actors only in 1 movie after top3 exclusion 

Create one hot encodings for a variety of datasets with different combinations of features
   
Keep movies that do not have any actors, directors, countries based on the above exclusions. Can still be recommended based on other attributes

__Exclude Rare Actors, Directors, Countries: Not Useful for Recommendations__    
And including all results in memory errors

In [46]:
# exclude if only in 1 movie (in dataset)
df.actors_lst = df.actors_lst.map(set) - set(actors_exclude)
df.director_lst = df.director_lst.map(set) - set(directors_exclude)
df.country_lst = df.country_lst.map(set) - set(country_exclude)
df.desc_top5 = df.desc_top5.map(set) - set(desc_exclude)
df.tag_top5 = df.tag_top5.map(set) - set(tags_exclude)
df.tags_rel = df.tags_rel.map(set) - set(tags_rel_exclude)
df.text_top5 = df.text_top5.map(set) - set(text_exclude)
df.production_company = np.where(df.production_company.isin(prod_exclude), np.nan, df.production_company)

# map back to list
df.actors_lst = df.actors_lst.map(list)
df.director_lst = df.director_lst.map(list)
df.country_lst = df.country_lst.map(list)
df.desc_top5 = df.desc_top5.map(list)
df.tag_top5 = df.tag_top5.map(list)
df.tags_rel = df.tags_rel.map(list)
df.text_top5 = df.text_top5.map(list)

__Get Top 3 actors from each movie based on appearances in all movies__            
If small movie where actors only appeared a few times, will still get those actors.   
For big movies, drop the background characters and focus on the big names    
    
3 hours to run

In [47]:
def get_top_actors(row, i):
    if len(row) >= i:
        return row[-i][0]
    else:
        return ''

In [49]:
# merge in number of movies each actor has been in based on actors_counts dataframe

#actors_explode = cat_list_expand(df, 'actors_lst')

#actors_movie_counts = actors_explode.actors_lst.apply(lambda row: actors_counts[actors_counts.actors == row].cnt.values[0]
#                                            ).to_frame()
#actors_movie_counts.columns = ['movie_cnt']
#actors_explode = pd.merge(actors_explode, actors_movie_counts, left_index = True, right_index = True)

In [50]:
# for each movie, sort by the number of appearances of each actor and keep the top 3 actors  
#actors_explode = actors_explode.sort_values(['movieId', 'movie_cnt'], ascending = False)
#actors_explode['actor_rank'] = 1
#actors_explode['actor_rank'] = actors_explode.groupby('movieId').actor_rank.cumsum()
#actors_explode = actors_explode[actors_explode.actor_rank <= 3]

# get list of unique actors for each movie 
#actors_explode = actors_explode.groupby('movieId').actors_lst.unique()

In [51]:
# convert to dataframe so compatible with parquet save
#actors_explode = actors_explode.to_frame()
#actors_explode.columns = ['actors_top3']
#actors_explode = actors_explode.actors_top3.map(list).to_frame()

In [48]:
#actors_explode.to_parquet('processed_files/top3_actors.parq', engine = 'fastparquet', compression = 'GZIP')
actors_explode = pd.read_parquet('processed_files/top3_actors.parq')

In [49]:
# merge top3 actors list with main movie dataframe
# OUTER merge so keep movies without any relevant actors 
actors_explode = actors_explode.actors_top3.map(list).to_frame()
df = pd.merge(df, actors_explode, left_on = 'movieId', right_index = True, how = 'outer')

In [50]:
# when missing after merge, replace to empty list rather than null
df.actors_top3 = df.actors_top3.apply(lambda d: d if isinstance(d, list) else [])

__Check again for actors in 1 movie__   
Now that have excluded some actors, once again look if any are in only 1 movie and thus not helpful for comparisons

In [51]:
# expand actor list
actors_expanded = cat_list_expand(df, 'actors_top3')

# count number of movies for each actor and prep for plotting 
actors_expanded = actors_expanded.groupby('actors_top3').actors_top3.count().to_frame()
actors_expanded.columns = ['cnt']
actors_expanded = actors_expanded.sort_values(['cnt'])
actors_expanded['actors'] = actors_expanded.index
actors_exclude_top3 = actors_expanded[actors_expanded.cnt == 1].actors.unique()

In [52]:
# exclude if only in 1 movie (in this processed dataset)
df.actors_top3 = df.actors_top3.map(set) - set(actors_exclude_top3)

# map back to list
df.actors_top3 = df.actors_top3.map(list)

## Create Dummies + Save
1. Dummies
2. Save parquet
3. Sparse + save npz   
   
Multiple versions with different combinations of metadata

### NOTE: Cannot run multiple versions in the same session, too much memory

In [55]:
def dummies_save(df_dum, var_lst, filename):
    
    # create dummies
    for var in var_lst:

        prefix = var.split('_')[0]

        # dummy variables
        dummies = pd.get_dummies(df_dum[var].apply(pd.Series).stack(), prefix = prefix).sum(level = 0)

        # merge back in
        # OUTER merge so that keep if don't have any entries in the dummies (empty list)
        df_dum = pd.merge(df_dum, dummies, left_index = True, right_index = True, how = 'outer')
                
            
    # drop unnecessary columns
    df_dum = df_dum.drop(columns = ['genre', 'genres', 'director', 'actors', 'country', 'genres_all', 'director_lst',
                                    'actors_lst', 'country_lst', 'actors_top3', 'desc_top5', 'description',
                                    'decade', 'production_company', 'tag_cleaned', 'tag_top5', 'tags_rel', 'text', 'text_top5'])
        
    # find dummy columns and replace NaN to 0 
    dum_cols = [i for i in df_dum.columns if not i in ['movieId', 'title_eng', 'year']]    
    for i in dum_cols:
        df_dum[i] = np.where(df_dum[i].isnull(), 0, df_dum[i])
    
    # save to parquet
    df_dum.to_parquet(filename + '.parq', engine = 'fastparquet', compression = 'GZIP')

    # convert to sparse and save
    cols = df_dum.columns # record original column names
    cols = list(cols[3:])
    movieIds = df_dum.movieId.to_list() # record movieIds corresponding to each row
    df_dum = df_dum.drop(columns = ['movieId', 'title_eng', 'year'])
    df_dum = scipy.sparse.csc_matrix(df_dum)

    scipy.sparse.save_npz(filename + "_sparse.npz", df_dum)
    
    # save row and column names with sparse data
    if filename == 'processed_files/processed_df':
        sparse_filename = 'processed_files/sparse_metadata'
    else:
        sparse_filename = 'processed_files/sparse_metadata' + filename.split('_df')[1] 

    with open(sparse_filename, "wb") as f:
        pickle.dump(cols, f)
        pickle.dump(movieIds, f)

In [56]:
#dummies_save(df, ['genres_all', 'actors_top3', 'director_lst'], 'processed_files/processed_df')

In [59]:
#dummies_save(df, ['desc_top5'], 'processed_files/processed_df_desc')

In [60]:
#dummies_save(df, ['genres_all', 'desc_top5'], 'processed_files/processed_df_desc_genre')

In [61]:
# get additional metadata but add onto genre, actor, director set. So load in, add back columns so works in function
#df_processed = pd.read_parquet('processed_files/processed_df.parq')
#df_processed = pd.merge(df_processed, df[['movieId', 'genre', 'genres', 'director', 'actors', 'country', 'genres_all', 
#                                          'director_lst', 'actors_lst', 'country_lst', 'actors_top3', 'desc_top5',
#                                          'description', 'decade', 'production_company', 'tag_cleaned', 'tag_top5', 
#                                         'tags_rel', 'text', 'text_top5']], on = 'movieId')
#del df
#dummies_save(df_processed, ['country_lst', 'decade', 'production_company'], 'processed_files/processed_df_all_meta')

In [62]:
# get additional metadata but add onto genre, actor, director set. So load in, add back columns so works in function
#df_processed = pd.read_parquet('processed_files/processed_df.parq')
#df_processed = pd.merge(df_processed, df[['movieId', 'genre', 'genres', 'director', 'actors', 'country', 'genres_all', 
#                                          'director_lst', 'actors_lst', 'country_lst', 'actors_top3', 'desc_top5',
#                                          'description', 'decade', 'production_company', 'tag_cleaned', 'tag_top5', 
#                                         'tags_rel', 'text', 'text_top5']], on = 'movieId')
#del df
#dummies_save(df_processed, ['tags_rel'], 'processed_files/processed_df_baseline_tags_rel')

In [63]:
#dummies_save(df, ['tag_top5'], 'processed_files/processed_df_tags')

In [64]:
#dummies_save(df, ['tags_rel'], 'processed_files/processed_df_tags_rel')

In [65]:
#dummies_save(df, ['text_top5'], 'processed_files/processed_df_text')

In [66]:
# save movie Ids of movies with tags and movies without tags
#with open('processed_files/movieIds_tags', "wb") as f:
#    pickle.dump(df[df.tag_cleaned != ''].movieId.unique(), f)
#with open('processed_files/movieIds_notags', "wb") as f:
#    pickle.dump(df[df.tag_cleaned == ''].movieId.unique(), f)